In [12]:
import pandas as pd # type: ignore
from sklearn.feature_extraction.text import TfidfVectorizer # type: ignore
from sklearn.metrics.pairwise import cosine_similarity # type: ignore
import mysql.connector


# Database connection
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="music_recommendation"
)

cursor = db.cursor()

def fetch_all_songs():
    query = """
    SELECT songs.song_id, songs.title, songs.artist, songs.genre, songs.emotion from songs 
    """
    cursor.execute(query)
    return cursor.fetchall()

data = fetch_all_songs()

# Sample data (replace with your dataset)
# data = {
#     'song_title': ['Song1', 'Song2', 'Song3', 'Song4', 'Song5'],
#     'artist': ['ram', 'ram', 'bhim', 'raaa', 'cam'],
#     'genre': ['Pop', 'Rock', 'Pop', 'Electronic', 'Rock'],
#     'emotion': ['Happy', 'Sad', 'sad', 'Happy', 'Sad']
# }

df = pd.DataFrame(data, columns=['id', 'song_title', 'artist', 'genre','emotion'])

df.head()

# df = pd.DataFrame(data)

# # User profile with multiple preferences
# user_profile = {
#     'favorite_artists': ['ram', 'raaa'],
#     'preferred_genres': ['Pop', 'Rock']
# }
def fetch_user_preference(user_id):
    query = """
    SELECT DISTINCT s.artist, s.genre FROM user_likes ul JOIN songs s ON ul.song_id = s.song_id WHERE ul.user_id = %s;
    """
    cursor.execute(query,(user_id,))
    return cursor.fetchall()

user_liked_data = fetch_user_preference(3)
user_profile = pd.DataFrame(user_liked_data, columns=['favorite_artists','preferred_genres'])
user_profile.head()

# Detected emotion
detected_emotion = 'love'

# Step 1: Filter songs by current emotion
filtered_df = df[df['emotion'].str.lower() == detected_emotion.lower()]

# Step 2: Create TF-IDF Vectorizer for genres, artists
tfidf = TfidfVectorizer(stop_words='english')
features = tfidf.fit_transform(filtered_df['genre'] + ' ' + filtered_df['artist'])

# Step 3: Compute similarity matrix (cosine similarity)
cosine_sim = cosine_similarity(features, features)

# Function to recommend songs based on user profile and detected emotion
def recommend_songs(user_profile, cosine_sim=cosine_sim, filtered_df=filtered_df):
    # Extract user preferences
    favorite_artists = user_profile['favorite_artists']
    preferred_genres = user_profile['preferred_genres']

    # Initialize an empty list to store results
    results = []

    # Iterate over each song in filtered_df
    for idx, row in filtered_df.iterrows():
        # Calculate similarity scores with user preferences
        artist_sim = 1 if row['artist'] in favorite_artists else 0
        genre_sim = 1 if row['genre'] in preferred_genres else 0
        similarity_score = (artist_sim + genre_sim) / 2  # Average similarity score

        # Add the song title, artist, and similarity scores to results
        results.append({
            'song_title': row['song_title'],
            'artist': row['artist'],
            'similarity_score': similarity_score
        })

    # Sort results by similarity score in descending order
    results = sorted(results, key=lambda x: x['similarity_score'], reverse=True)

    return results

# Example usage:
recommendations = recommend_songs(user_profile)

# Display top 3 recommended songs
print(f"\nTop 5 Recommended songs for user profile with detected emotion '{detected_emotion}':\n")
print("{:<15} {:<15} {:<20}".format('Song', 'Artist', 'Similarity Score'))
print("="*50)
for result in recommendations[:5]:  # Print only the top 3 recommendations
    print("{:<15} {:<15} {:<20.4f}".format(result['song_title'], result['artist'], result['similarity_score']))


Top 5 Recommended songs for user profile with detected emotion 'love':

Song            Artist          Similarity Score    
Senorita        Billie Eilish   0.0000              
Memories        Katy Perry      0.0000              
Circles         Shawn Mendes    0.0000              
Halo            The Beatles     0.0000              
Halo            Billie Eilish   0.0000              


In [32]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import mysql.connector

# Database connection
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="music_recommendation"
)

cursor = db.cursor()

# Function to fetch all songs from the database
def fetch_all_songs():
    query = """
    SELECT songs.song_id, songs.title, songs.artist, songs.genre, songs.emotion from songs 
    """
    cursor.execute(query)
    return cursor.fetchall()

# Fetch all songs data
data = fetch_all_songs()

# Convert data to DataFrame
df = pd.DataFrame(data, columns=['id', 'song_title', 'artist', 'genre', 'emotion'])
print(df.head())

# Function to fetch user preferences based on user_id
def fetch_user_preference(user_id):
    query = """
    SELECT DISTINCT s.artist, s.genre FROM user_likes ul 
    JOIN songs s ON ul.song_id = s.song_id 
    WHERE ul.user_id = %s;
    """
    cursor.execute(query, (user_id,))
    return cursor.fetchall()

# Fetch user liked data for a specific user (user_id=1 for example)
user_liked_data = fetch_user_preference(2)
user_profile = pd.DataFrame(user_liked_data, columns=['favorite_artists', 'preferred_genres'])
print(user_profile.head())

# Detected emotion (example)
detected_emotion = 'joy'

# Step 1: Filter songs by current emotion
filtered_df = df[df['emotion'].str.lower() == detected_emotion.lower()]

# Step 2: Create TF-IDF Vectorizer for genres, artists
tfidf = TfidfVectorizer(stop_words='english')
features = tfidf.fit_transform(filtered_df['genre'] + ' ' + filtered_df['artist'])

# Step 3: Compute similarity matrix (cosine similarity)
cosine_sim = cosine_similarity(features, features)

# Function to recommend songs based on user profile and detected emotion
def recommend_songs(user_profile, filtered_df, cosine_sim):
    # Extract user preferences
    favorite_artists = user_profile['favorite_artists'].tolist()
    preferred_genres = user_profile['preferred_genres'].tolist()

    # Initialize an empty list to store results
    results = []

    # Iterate over each song in filtered_df
    for idx, row in filtered_df.iterrows():
        # Calculate similarity scores with user preferences
        artist_sim = 1 if row['artist'] in favorite_artists else 0
        genre_sim = 1 if row['genre'] in preferred_genres else 0
        similarity_score = (artist_sim + genre_sim) / 2  # Average similarity score

        # Add the song title, artist, and similarity scores to results
        results.append({
            'song_title': row['song_title'],
            'artist': row['artist'],
            'similarity_score': similarity_score
        })

    # Sort results by similarity score in descending order
    results = sorted(results, key=lambda x: x['similarity_score'], reverse=True)

    # If no matching preferences, return any three songs based on current emotion
    if all(result['similarity_score'] == 0 for result in results):
        results = [{'song_title': row['song_title'], 'artist': row['artist'], 'similarity_score': 0} 
                   for idx, row in filtered_df.iterrows()]

    return results

# Example usage:
recommendations = recommend_songs(user_profile, filtered_df, cosine_sim)

# Display top 3 recommended songs
print(f"\nTop 3 Recommended songs for user profile with detected emotion '{detected_emotion}':\n")
print("{:<15} {:<15} {:<20}".format('Song', 'Artist', 'Similarity Score'))
print("="*50)
for result in recommendations[:3]:  # Print only the top 3 recommendations
    print("{:<15} {:<15} {:<20.4f}".format(result['song_title'], result['artist'], result['similarity_score']))


   id     song_title         artist    genre  emotion
0   1       Senorita       Dua Lipa      Pop     fear
1   2  Old Town Road            Sia      Pop  sadness
2   3  Old Town Road       Maroon 5      Pop    anger
3   4       Senorita  Billie Eilish     Rock     love
4   5        Circles         Eminem  Hip-Hop      joy
  favorite_artists preferred_genres
0         Dua Lipa              Pop
1              Sia              Pop
2         Maroon 5              Pop
3    Billie Eilish             Rock
4           Eminem          Hip-Hop

Top 3 Recommended songs for user profile with detected emotion 'joy':

Song            Artist          Similarity Score    
Circles         Eminem          1.0000              
Blinding Lights Dua Lipa        1.0000              
Bad Guy         Maroon 5        1.0000              


In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import mysql.connector

# Database connection
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="music_recommendation"
)

cursor = db.cursor()

# Function to fetch all songs from the database
def fetch_all_songs():
    query = """
    SELECT songs.song_id, songs.title, songs.artist, songs.genre, songs.emotion from songs 
    """
    cursor.execute(query)
    return cursor.fetchall()

# Fetch all songs data
data = fetch_all_songs()

# Convert data to DataFrame
df = pd.DataFrame(data, columns=['id', 'song_title', 'artist', 'genre', 'emotion'])
# print(df.head())

# Function to fetch user preferences based on user_id
def fetch_user_preference(user_id):
    query = """
    SELECT DISTINCT s.artist, s.genre FROM user_likes ul 
    JOIN songs s ON ul.song_id = s.song_id 
    WHERE ul.user_id = %s;
    """
    cursor.execute(query, (user_id,))
    return cursor.fetchall()

# Fetch user liked data for a specific user (user_id=1 for example)
user_liked_data = fetch_user_preference(3)
user_profile = pd.DataFrame(user_liked_data, columns=['favorite_artists', 'preferred_genres'])
# print(user_profile.head())

# Detected emotion (example)
detected_emotion = 'surprise'

# Step 1: Filter songs by current emotion
filtered_df = df[df['emotion'].str.lower() == detected_emotion.lower()]

# Step 2: Create TF-IDF Vectorizer for genres, artists
tfidf = TfidfVectorizer(stop_words='english')
features = tfidf.fit_transform(filtered_df['genre'] + ' ' + filtered_df['artist'])

# Step 3: Compute similarity matrix (cosine similarity)
cosine_sim = cosine_similarity(features, features)

# Function to recommend songs based on user profile and detected emotion
def recommend_songs(user_profile, cosine_sim=cosine_sim, filtered_df=filtered_df):
    # Extract user preferences
    favorite_artists = user_profile['favorite_artists'].tolist()
    preferred_genres = user_profile['preferred_genres'].tolist()

    # Initialize an empty list to store results
    results = []

    # Iterate over each song in filtered_df
    for idx, row in filtered_df.iterrows():
        # Calculate similarity scores with user preferences
        artist_sim = 1 if row['artist'] in favorite_artists else 0
        genre_sim = 1 if row['genre'] in preferred_genres else 0
        similarity_score = (artist_sim + genre_sim) / 2  # Average similarity score


        # Add the song title, artist, and similarity scores to results
        results.append({
            'song_title': row['song_title'],
            'artist': row['artist'],
            'similarity_score': similarity_score,
        })

    # Sort results by similarity score in descending order
    results = sorted(results, key=lambda x: x['similarity_score'], reverse=True)

    return results

# Example usage:
recommendations = recommend_songs(user_profile)

# Display top 3 recommended songs
print(f"\nTop 3 Recommended songs for user profile with detected emotion '{detected_emotion}':\n")
print("{:<15} {:<15} {:<20}".format('Song', 'Artist', 'Similarity Score'))
print("="*50)
for result in recommendations[:5]:  # Print only the top 3 recommendations
    print("{:<15} {:<15} {:<20.4f}".format(result['song_title'], result['artist'], result['similarity_score']))



Top 3 Recommended songs for user profile with detected emotion 'surprise':

Song            Artist          Similarity Score    
Unchained Melody Eminem          0.0000              
Hello           Adele           0.0000              
Stay With Me    John Legend     0.0000              
Shape of You    John Lennon     0.0000              
Old Town Road   Taylor Swift    0.0000              
